In [1]:
from Functions_lab4 import*

In [2]:
#parameters
base_dense=8
batch_size=8
LR=0.0001
img_w,img_h,img_ch=256,256,1


#metrics list
metric_list=[dice_coef,"Precision",'Recall']

#loss functions list
loss_func='categorical_crossentropy'

In [3]:
def load_image(data_list,img_w,img_h,img_ch,mask=False):
    image_array=np.zeros((len(data_list),img_w,img_h,img_ch),dtype='float32')
    for i in range(len(data_list)):
        Img=cv2.imread(data_list[i],0)
        Img=cv2.resize(Img[:,:],(img_w,img_h))
        if mask is True:
            Img[Img==156]=1
            Img[Img==251]=2
            Img[Img>2]=0
            image_array[i,:,:,0]=Img
        else:
            Img=Img.reshape(img_w,img_h)/255
            image_array[i,:,:,0]=Img
            
    return image_array

In [4]:
#load the X_ray images

#first create the set with the dir info
x_train,y_train,x_test,y_test = create_data('Lab3','CT',0.8)

#then load the images thanks to the info
x_train = load_image(x_train,img_w,img_h,img_ch)
y_train = load_image(y_train,img_w,img_h,img_ch,mask=True)
x_test = load_image(x_test,img_w,img_h,img_ch)
y_test = load_image(y_test,img_w,img_h,img_ch,mask=True)

#the intensity and resize rescale are already done in load_data function

In [5]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(y_train,num_classes=3)

In [6]:
y_test=to_categorical(y_test, num_classes=3)

In [7]:
def conv_block(InputLayer,base_dense,BatchNorm=False):
    if BatchNorm ==True:
        conv1 = Conv2D(base_dense,(3,3),strides = (1,1), padding = "same")(InputLayer)
        batch1 = BatchNormalization()(conv1)
        act1 = Activation('relu')(batch1)
        conv2 = Conv2D(base_dense,(3,3),strides = (1,1), padding = "same")(act1)
        batch2 = BatchNormalization()(conv2)
        act2 = Activation('relu')(batch2)

    else:
        conv1 = Conv2D(base_dense,(3,3),strides = (1,1), padding = "same", activation="relu")(InputLayer)
        conv2 = Conv2D(base_dense,(3,3),strides = (1,1), padding = "same", activation="relu")(conv1)
    return conv2

In [8]:
def get_unet_categorical(base_dense,img_w,img_h,img_ch,dropout=False,dr=0.2):
    input_size = (img_w, img_h,img_ch)
    input_layer = Input(shape=input_size, name='input_layer')
    
    if dropout==True:
        conv1 = conv_block(input_layer,base_dense,BatchNorm=True)
        pool1 = MaxPooling2D((2,2))(conv1)
        pool1 = Dropout(dr)(pool1)

        conv2 = conv_block(pool1,base_dense*2,BatchNorm=True)
        pool2 = MaxPooling2D((2,2))(conv2)
        pool2 = Dropout(dr)(pool2)

        conv3 = conv_block(pool2,base_dense*4,BatchNorm=True)
        pool3 = MaxPooling2D((2,2))(conv3)
        pool3 = Dropout(dr)(pool3)

        conv4 = conv_block(pool3,base_dense*8,BatchNorm=True)
        pool4 = MaxPooling2D((2,2))(conv4)
        pool4 = Dropout(dr)(pool4)

        #middle
        convm = conv_block(pool4,base_dense*16,BatchNorm=True)

        #deconvolution
        deconv1 = Conv2DTranspose(base_dense*8,(3,3),strides=(2,2),padding="same",activation="relu")(convm)
        uconv1=concatenate([deconv1,conv4])
        uconv1=Dropout(dr)(uconv1)
        uconv1=conv_block(uconv1,base_dense*8,BatchNorm=True)

        deconv2 = Conv2DTranspose(base_dense*4,(3,3),strides=(2,2),padding="same",activation="relu")(uconv1)
        uconv2=concatenate([deconv2,conv3])
        uconv2=Dropout(dr)(uconv2)
        uconv2=conv_block(uconv2,base_dense*4,BatchNorm=True)

        deconv3= Conv2DTranspose(base_dense*2,(3,3),strides=(2,2),padding="same",activation="relu")(uconv2)
        uconv3=concatenate([deconv3,conv2])
        uconv3=Dropout(dr)(uconv3)
        uconv3=conv_block(uconv3,base_dense*2,BatchNorm=True)

        deconv4 = Conv2DTranspose(base_dense,(3,3),strides=(2,2),padding="same",activation="relu")(uconv3)
        uconv4=concatenate([deconv4,conv1])
        uconv4=Dropout(dr)(uconv4)
        uconv4=conv_block(uconv4,base_dense,BatchNorm=True)
    
    
    else:
        conv1 = conv_block(input_layer,base_dense,BatchNorm=True)
        pool1 = MaxPooling2D((2,2))(conv1)

        conv2 = conv_block(pool1,base_dense*2,BatchNorm=True)
        pool2 = MaxPooling2D((2,2))(conv2)

        conv3 = conv_block(pool2,base_dense*4,BatchNorm=True)
        pool3 = MaxPooling2D((2,2))(conv3)

        conv4 = conv_block(pool3,base_dense*8,BatchNorm=True)
        pool4 = MaxPooling2D((2,2))(conv4)

        #middle
        convm = conv_block(pool4,base_dense*16,BatchNorm=True)

        #deconvolution
        deconv1 = Conv2DTranspose(base_dense*8,(3,3),strides=(2,2),padding="same",activation="relu")(convm)
        uconv1=concatenate([deconv1,conv4])
        uconv1=conv_block(uconv1,base_dense*8,BatchNorm=True)

        deconv2 = Conv2DTranspose(base_dense*4,(3,3),strides=(2,2),padding="same",activation="relu")(uconv1)
        uconv2=concatenate([deconv2,conv3])
        uconv2=conv_block(uconv2,base_dense*4,BatchNorm=True)

        deconv3= Conv2DTranspose(base_dense*2,(3,3),strides=(2,2),padding="same",activation="relu")(uconv2)
        uconv3=concatenate([deconv3,conv2])
        uconv3=conv_block(uconv3,base_dense*2,BatchNorm=True)

        deconv4 = Conv2DTranspose(base_dense,(3,3),strides=(2,2),padding="same",activation="relu")(uconv3)
        uconv4=concatenate([deconv4,conv1])
        uconv4=conv_block(uconv4,base_dense,BatchNorm=True)
        
    output_layer=Conv2D(3,(1,1),padding='same',activation='softmax',name='output_layer')(uconv4)
    
    model=Model(inputs=input_layer, outputs=output_layer)
    model.summary()
    
    return model

In [9]:
my_model3=get_unet_categorical(base_dense,img_w,img_h,img_ch,dropout=True,dr=0.2)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 8)  80          input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_v2 (BatchNo (None, 256, 256, 8)  32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 8)  0           batch_normalization_v2[0][0]     
______________________________________________________________________________________________

In [10]:
#data augmentation
dictionary_augmentation = dict(
                     rotation_range=10,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,       
                     horizontal_flip=True)



In [11]:
train_generator, STEP_SIZE_TRAIN=augmentation(x_train,y_train,dictionary_augmentation,batch_size)

In [12]:
model_hist=compile_fit_generator(my_model3,train_generator,x_test,y_test,loss_func, optimizer=Adam(LR),metrics=dice_coef,batch_size=batch_size,n_epochs=80,STEP_SIZE_TRAIN=STEP_SIZE_TRAIN)

Epoch 1/80
1683/1683 [==============================] - 214s 127ms/step - loss: 0.3003 - dice_coef: 0.8033 - val_loss: 0.1684 - val_dice_coef: 0.8981
Epoch 2/80
1163/1683 [===================>..........] - ETA: 56s - loss: 0.1216 - dice_coef: 0.9268

KeyboardInterrupt: 